# Exercise 3 - Putting it all together - Model Evaluation against Validation Set

In this notebook we will use our train model to do some prediction on our validation set. In the competition, we will evaluate your model against unseen test set!

Here are the steps to evaluate your model:
1. Download the necessary Python module (AiCampEval) to prepare for submission 
2. Load your trained model
3. Write a function that takes in a list of numpy array images and returns a list of predictions
4. Pass the function into the evaluation model, enter your `team_id` and `submission_type`, and wait for your result

Let's get started!

## Download and Install the Evaluation Module

The following command downloads and installs the evaluation module required to do your submission. Note: If you fail to load the module `AiCampEval` after running `pip install`, please try restarting your kernel and run again.

In [ ]:
!pip install --force-reinstall --user --no-warn-script-location -q https://ai-camp.s3-us-west-2.amazonaws.com/AiCampEval-1.7-py3-none-any.whl

In [ ]:
from AiCampEval import eval_submit

## Load Model

Let's begin by loading our best performing model.

In [ ]:
from keras.models import load_model

model_path = 'saved_model.hdf5'
model = load_model( model_path )

Load in `train_generator` to get the class_indices as we will be using it in our function `evaluate_images`

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'data/trainset_11classes_0_00000'
train_folder = os.path.join(base_dir, 'train')

# Batch size
bs = 32

# All images will be resized to this value
image_size = (299, 299)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

print(train_generator.class_indices)

## Model Prediction against Test Set

Using the trained model, we now predict the classes in a **test set**. The test set contains images from all classes, all placed in a single folder. In this toy example, we will be using the `val` folder as a test set, so expect to see the same score as what we got for validation above.

You will be required to create a function that accepts a list of numpy array images as input and outputs a list of prediction strings corresponding to each image in the input list.

`predictions` should be a list of string: [ 'ChildPose', 'KoreanHeart', 'ChildPose', 'KungfuSalute', ...]. It's length should be the length of list_of_np_arrays.

In [ ]:
import numpy as np
from PIL import Image

'''
This is a helper function to preprocess the input before sending it through my network. 
It performs the following steps:
1. Resizes each image to (224,224,3), which are the image dimensions my model was trained on.
2. Normalizes the range of each pixel's value from [0-255] to [0-1].
3. Reshapes each array such that it has a batch dimension: from  (224,224,3) -> (1,224,224,3), 
because my model.predict function expects a batch dimension.

Implement your own preprocessing function according to your model's needs.
''' 
def preprocess_arrays(list_of_np_arrays):
    target_size = image_size
    pil_images = [ Image.fromarray(arr.astype('uint8'), 'RGB') for arr in list_of_np_arrays ]
    resized_pil_images = [img.resize( target_size, Image.NEAREST ) for img in pil_images]
    resized_images = [np.array( img ) for img in resized_pil_images]
    preprocessed_images = [np.expand_dims(x, axis=0) / 255. for x in resized_images]
    return preprocessed_images

'''
This function accepts a list of numpy array images as input and outputs a list of prediction strings 
corresponding to each image in the input list.
IMPORTANT: you have to implement this function.
''' 
def evaluate_images(list_of_np_arrays):
    # Swap the key and value in the class_indices
    label_dict = dict((v,k) for k,v in (train_generator.class_indices).items())
    # Run through the list_of_np_arrays to perform any preprocessing you need
    preprocessed_imgs = preprocess_arrays( list_of_np_arrays )
    # Get a list of softmax_vectors by passing in the preprocessed_imgs to model.predict()
    softmax_vectors = [ model.predict( x )[0] for x in preprocessed_imgs ]
    # Get the predicted_class_indices by running each of the softmax_vectors through np.argmax()
    predicted_class_indices = [ np.argmax( x ) for x in softmax_vectors ]
    # Convert the list of class_indices to a list of predictions (in str)
    # predictions is a list of labels: ['KoreanHeart', 'KoreanHeart','ChairPose',.......]
    predictions = [ label_dict[k] for k in predicted_class_indices ]
    return predictions

## Submission

The `eval_submit` function takes in 3 parameters - your function, TEAM_ID and SUBMISSION_TYPE

In [ ]:
## CHANGE HERE
TEAM_ID = "CHANGE_HERE"
SUBMISSION_TYPE = "trainset_11classes_0_val"

eval_submit(evaluate_images, SUBMISSION_TYPE, TEAM_ID )